In [1]:
# Importing necessary libraries
import requests
import cv2
import numpy as np
import os
import csv

In [2]:
# Downloading YOLOv3 weights
url_weights = "https://pjreddie.com/media/files/yolov3.weights"
response_weights = requests.get(url_weights)
with open("yolov3.weights", "wb") as f:
    f.write(response_weights.content)

# Downloading YOLOv3 configuration file
url_cfg = "https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg?raw=true"
response_cfg = requests.get(url_cfg)
with open("yolov3.cfg", "wb") as f:
    f.write(response_cfg.content)

# Downloading YOLOv3 class names
url_names = "https://github.com/pjreddie/darknet/blob/master/data/coco.names?raw=true"
response_names = requests.get(url_names)
with open("coco.names", "wb") as f:
    f.write(response_names.content)

In [3]:
# Loading YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Loading COCO class names
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [4]:
# Function for detecting objects
def detect_objects(img, net, output_layers):
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing information on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] == "car":
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    return boxes, confidences, class_ids, indexes

In [5]:
# Function for drawing bounding boxes and saving the image and CSV data
def draw_bounding_boxes_and_save(img, boxes, confidences, class_ids, indexes, output_path, img_name, csv_writer):
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)  # Green color for car
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, f"{label} {confidence:.2f}", (x, y + 30), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
            # Write bounding box information to CSV
            csv_writer.writerow([img_name, label, confidence, x, y, w, h])
    
    # Save the image
    save_path = os.path.join(output_path, img_name)
    cv2.imwrite(save_path, img)

# Function to initialize CSV writer
def initialize_csv_writer(csv_file_path):
    os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)
    file = open(csv_file_path, mode='w', newline='')
    writer = csv.writer(file)
    header = ["image_name", "label", "confidence", "x", "y", "width", "height"]
    writer.writerow(header)
    return file, writer

# Initialize CSV writers
training_csv_path = "output_training_images/training_boxes.csv"
test_csv_path = "output_test_images/test_boxes.csv"

training_csv_file, training_csv_writer = initialize_csv_writer(training_csv_path)
test_csv_file, test_csv_writer = initialize_csv_writer(test_csv_path)

In [6]:
# Processing training images
training_images_path = "training_images/"
training_images = [f for f in os.listdir(training_images_path) if os.path.isfile(os.path.join(training_images_path, f))]

for img_name in training_images:
    img_path = os.path.join(training_images_path, img_name)
    img = cv2.imread(img_path)
    boxes, confidences, class_ids, indexes = detect_objects(img, net, output_layers)
    draw_bounding_boxes_and_save(img, boxes, confidences, class_ids, indexes, "output_training_images/", img_name, training_csv_writer)

In [7]:
# Processing test images
test_images_path = "testing_images/"
test_images = [f for f in os.listdir(test_images_path) if os.path.isfile(os.path.join(test_images_path, f))]

for img_name in test_images:
    img_path = os.path.join(test_images_path, img_name)
    img = cv2.imread(img_path)
    boxes, confidences, class_ids, indexes = detect_objects(img, net, output_layers)
    draw_bounding_boxes_and_save(img, boxes, confidences, class_ids, indexes, "output_test_images/", img_name, test_csv_writer)

In [8]:
training_csv_file.close()
test_csv_file.close()